In [1]:
import numpy as np
import pandas as pd
from random import sample

from coremdlr.config import defaults
from coremdlr.datasets.utils import infer_test_wells

from coremdlr.models import FeaturePredictor, LambdaModel
from coremdlr.datasets import WellLoader, FaciesDataset

Using TensorFlow backend.


In [2]:
import hyperopt
from hyperopt import hp
from hyperopt.pyll.base import scope
from sklearn.metrics import f1_score, log_loss
from sklearn.utils.class_weight import compute_sample_weight

In [3]:
fmodel_args = {
    'logs': {
        'model': 'LambdaModel',
        'model_args': {'feature': 'logs'}
    }
}

XGB_SEARCH_SPACE = {
    'model_type' : 'XGB',
    'max_depth' : scope.int(hp.quniform('max_depth', 3, 10, 1)),
    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators' : scope.int(hp.quniform('n_estimators', 10, 1000, 1)),
    'objective' : 'multi:softprob',
    'n_jobs' : 2,
    'gamma' : hp.uniform('gamma', 0, 0.5),
    'subsample' : hp.uniform('subsample', 0.3, 1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.3, 1.0),
    'colsample_bylevel' : 1,
    'reg_alpha' : 0,                                    # L1 penalty
    'reg_lambda' : hp.uniform('reg_lambda', 0.1, 10),   # L2 penalty
    'tree_method' : 'gpu_exact',
}

def make_params_usable(params):
    """
    Convert params returned by hyperopt to usable `model_args`
    """
    return {
        **XGB_SEARCH_SPACE,
        **params,
        **{'max_depth' : int(params['max_depth']),
           'n_estimators' : int(params['n_estimators'])}
    }

In [4]:
def run_experiment(num_train, max_evals=10, search_space=XGB_SEARCH_SPACE):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    datasets = [FaciesDataset(s, infer_test_wells(s), features=['logs']) for s in train_sets]
    
    for dset in datasets:
        dset.load_or_generate_data()
    
    def train_model(model_config):
        
        log_losses = []
        
        for dset in datasets:
            
            model = FeaturePredictor(dset, model_args=model_config, feature_model_args=fmodel_args)
            model.fit(dset, verbose=False)
            print('F1 score:', f1_score(dset.y_test, model.predict(dset.X_test), average='macro'))
            
            log_losses.append(log_loss(dset.y_test, model.predict_proba(dset.X_test), 
                                      sample_weight=compute_sample_weight('balanced', dset.y_test)))
            
        return np.mean(log_losses)
    
    return hyperopt.fmin(
        fn=train_model,
        space=XGB_SEARCH_SPACE,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals
    )

In [16]:
best_params_3 = run_experiment(3)

print('\n\n\t Starting TRAIN_SET_6')
best_params_6 = run_experiment(6)

print('\n\n\t Starting TRAIN_SET_9')
best_params_9 = run_experiment(9)

Loading Well:  204-24a-7  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-24a-6  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('logs', (13006, 11))]
Loading Well:  204-20-3  from  /home/administrator/Dropbox/core_data/facies/train_data


...

Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('logs', (3842, 11))]
Loading Well:  204-19-7  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9807,)), ('top', (9807,)), ('base', (9807,)), ('logs', (9807, 11))]
Loading Well:  204-19-3A  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('logs', (9590, 11))]
Training model for feature:  logs
F1 score: 0.14166930426540317
Training model for feature:  logs
F1 score: 0.23874602518279583
Training model for feature:  logs
F1 score: 0.27821915026592386
Training model for feature:  logs
F1 score: 0.1529223915629262
Training model for feature:  logs
F1 score: 0.2397779160513009
Training model for feature:  logs
F1 score: 0.2791934498988608
Training model for feature:  logs
F1 score: 0.13986143408448087
Training model for feature:  logs
F1 score: 0.24848473361324844
Training model for feature:  logs
F1 score: 0.2695345637559738
Training model for feature:  logs
F1 score: 0.12818314303890171
Training model for feature:  logs
F1 score: 0.29620068686603984
Training model for feature:  logs
F1 score: 0.2491172092291932
Training model for feature:  logs
F1 score: 0.1459216458067332
Training model for feature:  logs
F1 score: 0.2458823847

In [5]:
best_params_3 = {'colsample_bytree': 0.715782636188357,
 'gamma': 0.38031439765997715,
 'learning_rate': 0.13285585695446292,
 'max_depth': 3.0,
 'n_estimators': 107.0,
 'reg_lambda': 7.063383010063417,
 'subsample': 0.5627460184070672}

In [6]:
best_params_6 = {'colsample_bytree': 0.5044540258165737,
 'gamma': 0.3606833915563988,
 'learning_rate': 0.020567898195521434,
 'max_depth': 8.0,
 'n_estimators': 278.0,
 'reg_lambda': 5.316977094203006,
 'subsample': 0.6810694533283663}

In [7]:
best_params_9 = {'colsample_bytree': 0.4845409209422611,
 'gamma': 0.4368302424155825,
 'learning_rate': 0.11360839815510773,
 'max_depth': 3.0,
 'n_estimators': 90.0,
 'reg_lambda': 8.283549867329606,
 'subsample': 0.9353591985992722}

In [8]:
def pred_results_df(num_train, model_config):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    datasets = [FaciesDataset(s, infer_test_wells(s), features=['logs']) for s in train_sets]
    
    pred_dfs = []
    
    for i, dset in enumerate(datasets):
        dset.load_or_generate_data()
        
        model = FeaturePredictor(dset, model_args=model_config, feature_model_args=fmodel_args)
        model.fit(dset, verbose=False)
        
        print('predict_proba: ', model.predict_proba(dset.X_test))
        
        for test_well_name in dset.test_well_names:
            df = model.preds_dataframe(test_well_name, logs=dset.logs_args['which_logs'])
            df['well'] = test_well_name
            df['set_number'] = i
            pred_dfs.append(df)
        
    return pd.concat(pred_dfs)

In [9]:
preds_df_3 = pred_results_df(3, make_params_usable(best_params_3))
preds_df_3.to_csv('logs_xgb_results_train_sets3.csv')

Loading Well:  204-24a-7  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-24a-6  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('logs', (13006, 11))]
Loading Well:  204-20-3  from  /home/administrator/Dropbox/core_data/facies/train_data


...

Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('logs', (3842, 11))]
Loading Well:  204-20-3  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('logs', (10906, 11))]
Loading Well:  204-20a-7  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (3105,)), ('top', (3105,)), ('base', (3105,)), ('logs', (3105, 11))]
Training model for feature:  logs
predict_proba:  [[0.130104   0.02122485 0.22306114 0.22919868 0.39641136]
 [0.130104   0.02122485 0.22306114 0.22919868 0.39641136]
 [0.12992227 0.02259199 0.22274958 0.22887853 0.39585763]
 ...
 [0.0219544  0.02598689 0.0672323  0.24206477 0.64276165]
 [0.0219544  0.02598689 0.0672323  0.24206477 0.64276165]
 [0.0219544  0.02598689 0.0672323  0.24206477 0.64276165]]


In [10]:
preds_df_6 = pred_results_df(6, make_params_usable(best_params_6))
preds_df_6.to_csv('logs_xgb_results_train_sets6.csv')

Loading Well:  204-24a-7  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-24a-6  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('logs', (13006, 11))]
Loading Well:  204-20-3  from  /home/administrator/Dropbox/core_data/facies/train_data


...

Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (1917,)), ('top', (1917,)), ('base', (1917,)), ('logs', (1917, 11))]
Loading Well:  204-19-3A  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('logs', (9590, 11))]
Loading Well:  204-24a-7  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Training model for feature:  logs
predict_proba:  [[0.5178043  0.25870684 0.04160369 0.02629956 0.15558562]
 [0.5149263  0.2509222  0.0403518  0.02550819 0.1682915 ]
 [0.5149263  0.2509222  0.0403518  0.02550819 0.1682915 ]
 ...
 [0.08460303 0.65295815 0.20981815 0.01960625 0.0330144 ]
 [0.08460303 0.65295815 0.20981815 0.01960625 0.0330144 ]
 [0.08460303 0.65295815 0.20981815 0.01960625 0.0330144 ]]


In [11]:
preds_df_9 = pred_results_df(9, make_params_usable(best_params_9))
preds_df_9.to_csv('logs_xgb_results_train_sets9.csv')

Loading Well:  204-19-3A  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('logs', (9590, 11))]
Loading Well:  204-20-1  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('logs', (11210, 11))]
Loading Well:  204-19-7  from  /home/administrator/Dropbox/core_data/facies/train_data


...

Adding NaN log:  DTS
Feature shapes:  [('depth', (9807,)), ('top', (9807,)), ('base', (9807,)), ('logs', (9807, 11))]
Loading Well:  204-19-3A  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('logs', (9590, 11))]
Loading Well:  205-21b-3  from  /home/administrator/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('logs', (3842, 11))]
Training model for feature:  logs
predict_proba:  [[0.0905811  0.06074808 0.12568405 0.04368788 0.6792989 ]
 [0.09108742 0.05549799 0.12638658 0.04393208 0.68309593]
 [0.08108775 0.03822483 0.14118522 0.03910917 0.700393  ]
 ...
 [0.24134491 0.03218095 0.01027599 0.66905975 0.04713842]
 [0.24134491 0.03218095 0.01027599 0.66905975 0.04713842]
 [0.24134491 0.03218095 0.01027599 0.66905975 0.04713842]]


In [12]:
preds_df_3.shape, preds_df_6.shape, preds_df_9.shape

((152880, 25), (108025, 25), (71560, 25))

In [13]:
preds_df_3.head()

,base,confidence,depth,proba_0,proba_1,proba_2,proba_3,proba_4,regression,top,...,NEUT,PEF,RDEP,RSHAL,DTC,DTS,DTS1,DTS2,well,set_number
0,2208.085920,0.558097,2208.083152,0.162600,0.080687,0.558097,0.190883,0.007732,1.660552,2208.080550,...,-1.084784,-1.730394,1.346995,1.233757,1.417721,0.0,-0.148382,-0.483111,204-19-6,0
1,2208.091290,0.557830,2208.088522,0.162522,0.080648,0.557830,0.190792,0.008208,1.661818,2208.085920,...,-1.104229,-1.742244,1.349403,1.233453,1.421162,0.0,-0.136087,-0.474946,204-19-6,0
2,2208.096661,0.596532,2208.093892,0.157963,0.075003,0.596532,0.162869,0.007634,1.644650,2208.091290,...,-1.123674,-1.754094,1.351810,1.233149,1.424602,0.0,-0.123793,-0.466782,204-19-6,0
3,2208.102031,0.596227,2208.099262,0.158082,0.075059,0.596227,0.162992,0.007639,1.644456,2208.096661,...,-1.142953,-1.765778,1.354180,1.232846,1.428043,0.0,-0.111576,-0.458599,204-19-6,0
4,2208.107401,0.497442,2208.104632,0.193944,0.099274,0.497442,0.199968,0.009372,1.577861,2208.102031,...,-1.147980,-1.763182,1.353276,1.232685,1.431540,0.0,-0.106107,-0.448906,204-19-6,0


In [14]:
preds_df_3.columns

Index(['base', 'confidence', 'depth', 'proba_0', 'proba_1', 'proba_2',
       'proba_3', 'proba_4', 'regression', 'top', 'y_pred', 'y_true', 'GR',
       'SP', 'DENS', 'NEUT', 'PEF', 'RDEP', 'RSHAL', 'DTC', 'DTS', 'DTS1',
       'DTS2', 'well', 'set_number'],
      dtype='object')